In [1]:
!pip install datasets transformers evaluate 
!pip install speechbrain==0.5.16 faster-whisper pyannote.audio 
!pip install whisper ctranslate2==4.4.0 librosa soundfile scikit-learn
!pip install librosa torch torchaudio numpy pandas matplotlib tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 2.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of pyannote-audio to determine which version is compatible with other requirements. This could take a while.
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 630.6/630.6 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.8/38.8 MB 47.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 48.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/indic-tts-deepfake-challenge/sample.csv


In [4]:
# Import necessary libraries
import os
import librosa
import numpy as np
import pandas as pd
import soundfile as sf
from datetime import timedelta
from faster_whisper import WhisperModel
from pyannote.audio import Inference
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from speechbrain.pretrained import SpeakerRecognition
from dataclasses import dataclass
from typing import Any, Dict, List, Optional, Union


ModuleNotFoundError: No module named 'faster_whisper'

In [5]:
from datasets import load_dataset

# Official dataset for this challenge
dataset = load_dataset("SherryT997/IndicTTS-Deepfake-Challenge-Data")  

# Train and test splits
train_data = dataset["train"]  # Contains 'is_tts' labels
test_data = dataset["test"]  # 'is_tts' is -1 for all rows

README.md:   0%|          | 0.00/2.81k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/35 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/35 [00:00<?, ?it/s]

train-00000-of-00035.parquet:   0%|          | 0.00/453M [00:00<?, ?B/s]

train-00001-of-00035.parquet:   0%|          | 0.00/461M [00:00<?, ?B/s]

...

test-00002-of-00004.parquet:   0%|          | 0.00/410M [00:00<?, ?B/s]

test-00003-of-00004.parquet:   0%|          | 0.00/291M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/31102 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2635 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/35 [00:00<?, ?it/s]

In [6]:
train_data

Dataset({
    features: ['text', 'id', 'language', 'is_tts', 'audio'],
    num_rows: 31102
})

In [7]:
train_data[0]

{'text': 'বিত্তীয় সময়সীমা মানি চলাত বাৰে বাৰে বিফল হোৱাৰ বাবে মই বৰ হতাশ হৈছোঁ।',
 'id': 'ASM_F_ANGER_00342',
 'language': 'Assamese',
 'is_tts': 1,
 'audio': {'path': 'ASM_F_ANGER_00342.wav',
  'array': array([-0.00297868, -0.00474697, -0.00408163, ..., -0.00600238,
         -0.00075703,  0.00277057]),
  'sampling_rate': 16000}}

In [8]:
print(train_data[0:5]['is_tts'])
print(test_data[0:5]['is_tts'])

[1, 0, 0, 1, 0]
[-1, -1, -1, -1, -1]


In [8]:
# Filter only required columns
#train_data_1 = train_data.remove_columns([col for col in train_data.column_names if col not in ["text", "is_tts", "audio"]])
#test_data_1 = test_data.remove_columns([col for col in test_data.column_names if col not in ["text", "audio"]])  # No is_tts


In [9]:
shuffled_dataset = train_data.shuffle(seed=42)

In [10]:
train_df = shuffled_dataset.select(range(1000))

In [11]:
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor, AutoModelForCTC, TrainingArguments, Trainer
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h", is_tokenizer=False)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [12]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def prepare_dataset(batch):
    audio = batch["audio"]

    # Extract features from audio
    batch["input_values"] = processor(
        audio["array"],  # Correct variable name
        sampling_rate=16000,
        return_tensors="pt",
        padding=True
    )['input_values'][0].to(device)  # Move to device

    batch["input_length"] = len(batch["input_values"])
    batch["labels"] = batch["is_tts"]  # Ensure labels are binary (0 or 1)

    return batch


In [13]:
dataset = train_df.map(prepare_dataset, batch_size=8)
dataset

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'id', 'language', 'is_tts', 'audio', 'input_values', 'input_length', 'labels'],
    num_rows: 1000
})

In [25]:
@dataclass
class DataCollatorWithPadding:
    """
    Data collator that dynamically pads inputs but keeps labels as-is.
    """
    processor: Wav2Vec2Processor  # Use full processor for consistency
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # Extract input features (waveforms)
        input_features = [{"input_values": feature["input_values"]} for feature in features]

        # Apply padding using processor
        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )

        # Convert labels to tensor
        batch["labels"] = torch.tensor([feature["labels"] for feature in features], dtype=torch.long)

        return batch

# ✅ Use processor directly
data_collator = DataCollatorWithPadding(processor=processor, padding=True)

In [26]:
pip install evaluate

Note: you may need to restart the kernel to use updated packages.


In [27]:
import numpy as np
import torch
from evaluate import load

from scipy.special import softmax  # Ensures outputs are probabilities

# Load evaluation metrics
accuracy_metric = load("accuracy")
precision_metric = load("precision")
recall_metric = load("recall")
f1_metric = load("f1")
roc_auc_metric = load("roc_auc")

def compute_metrics(pred):
    # Extract logits from model predictions
    pred_logits = pred.predictions
    
    # Convert logits to probabilities using softmax
    pred_probs = softmax(pred_logits, axis=-1)[:, 1]  # Probability of class 1 (synthetic speech)

    # Get true labels
    labels = pred.label_ids

    # Compute metrics
    accuracy = accuracy_metric.compute(predictions=pred_probs.round(), references=labels)["accuracy"]
    precision = precision_metric.compute(predictions=pred_probs.round(), references=labels, average="binary")["precision"]
    recall = recall_metric.compute(predictions=pred_probs.round(), references=labels, average="binary")["recall"]
    f1 = f1_metric.compute(predictions=pred_probs.round(), references=labels, average="binary")["f1"]
    roc_auc = roc_auc_metric.compute(prediction_scores=pred_probs, references=labels)["roc_auc"]

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "roc_auc": roc_auc  # Important for evaluating probability-based outputs
    }


In [28]:
dataset1 = dataset.train_test_split(test_size=0.1, shuffle=True, seed=42)

In [35]:
from transformers import AutoModelForAudioClassification

model = AutoModelForAudioClassification.from_pretrained(
    "facebook/wav2vec2-base",
    num_labels=2  # Binary classification (0: synthetic, 1: real)
)
model.to("cuda")


Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Wav2Vec2ForSequenceClassification(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)


In [36]:
model.freeze_feature_encoder()

In [37]:
training_args = TrainingArguments(
  output_dir="AsrTaskModel",
  group_by_length=True,
  per_device_train_batch_size=8,
  per_device_eval_batch_size=8,
  evaluation_strategy="steps",
  num_train_epochs=7,
  fp16=True,
  gradient_checkpointing=True,
  save_steps=1000,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
  load_best_model_at_end=True,
  save_strategy="steps",
  report_to="none" # put this, else it will ask for wandb api
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [38]:
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset1["train"],
    eval_dataset=dataset1["test"],
    tokenizer=processor,  # Add this if using Wav2Vec2Processor
)


<ipython-input-38-3b020d1ba6fc>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [39]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Roc Auc
500,0.340900,0.079508,0.980000,0.976744,0.976744,0.976744,0.998776


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Could not locate the best model at AsrTaskModel/checkpoint-500/pytorch_model.bin, if you are running a distributed training on multiple nodes, you should activate `--save_on_each_node`.


TrainOutput(global_step=791, training_loss=0.26649892254816143, metrics={'train_runtime': 675.1436, 'train_samples_per_second': 9.331, 'train_steps_per_second': 1.172, 'total_flos': 3.950535354261249e+17, 'train_loss': 0.26649892254816143, 'epoch': 7.0})

In [2]:
test_data

NameError: name 'test_data' is not defined

In [1]:
import torch
import pandas as pd
import numpy as np
from scipy.special import softmax
from tqdm import tqdm

# Ensure model is in evaluation mode
model.eval()

# Prepare a list to store submission results
submission_results = []

# Process test dataset
for sample in tqdm(test_data):
    sample_id = sample["id"]
    audio_input = sample["audio"]["array"]

    # Process audio input
    inputs = processor(audio_input, sampling_rate=16000, return_tensors="pt", padding=True)

    # Move to GPU if available
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Run inference
    with torch.no_grad():
        logits = model(**inputs).logits.cpu().numpy()

    # Convert logits to probabilities
    probabilities = softmax(logits, axis=-1)
    
    # Probability of being synthetic (is_tts = 1)
    is_tts_prob = probabilities[0, 1]

    # Append to submission list
    submission_results.append({"id": sample_id, "is_tts": is_tts_prob})

# Convert to DataFrame
submission_df = pd.DataFrame(submission_results)

# Save to CSV
submission_df.to_csv("submission.csv", index=False)

print("✅ Submission file saved as 'submission.csv'")


NameError: name 'model' is not defined

In [ ]:
import pandas as pd
import torch
from scipy.special import softmax

# Move model to evaluation mode
model.eval()

# Ensure the model is on the right device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Create a list to store predictions
predictions = []

# Iterate over the test dataset
for batch in test_dataloader:  # Assuming `test_dataloader` is your test dataset loader
    input_values = batch["input_values"].to(device)

    # Forward pass
    with torch.no_grad():
        logits = model(input_values).logits

    # Convert logits to probabilities using softmax
    probs = softmax(logits.cpu().numpy(), axis=-1)[:, 1]  # Probability of class 1 (synthetic)

    # Store predictions with IDs
    for idx, prob in zip(batch["id"].cpu().numpy(), probs):
        predictions.append({"id": int(idx), "is_tts": round(float(prob), 3)})  # Round to 3 decimal places

# Convert to DataFrame
submission_df = pd.DataFrame(predictions)

# Save as CSV
submission_df.to_csv("submission.csv", index=False)

print("✅ Submission file saved as submission.csv")
